In [1]:
import summarize
import sentences
import nltk

import numpy as np
import pandas as pd

In [2]:
# Load the Data
data = pd.read_csv('data.csv')
data.head(2)

,X.AUTHID,anger_doc_sum,anticipation_doc_sum,arousal_doc_sum,disgust_doc_sum,dominance_doc_sum,fear_doc_sum,joy_doc_sum,neg_doc_sum,num_text_words,...,cSIN_label_fin,cEXC_label_fin,cCOM_label_fin,cRUG_label_fin,cSOP_label_fin,cSIN_fin_rank,cEXC_fin_rank,cCOM_fin_rank,cRUG_fin_rank,cSOP_fin_rank
0,b'http://blog.searsholdings.com/leadership-vie...,1,9,31.317,0,38.293,7,16,7,124,...,n,y,y,n,y,5.0,1.0,2.0,3.0,4.0
1,b'https://atyourservice.blogs.xerox.com/catego...,0,4,23.260,0,32.665,0,1,2,126,...,y,n,y,n,y,NaN,NaN,NaN,NaN,NaN


In [3]:
## Process into documents
site_content = data['site.content']
docs = []
for i in range(site_content.size):
    docs.append(eval(site_content[i]).decode('utf-8'))

In [4]:
## Split into sentences
sent_split_docs = []
for doc in docs:
    sent_split_docs.append(nltk.sent_tokenize(doc)) 

In [20]:
## Feature based summarisation

feature_base_sents=[]
for d,doc in enumerate(sent_split_docs):
    
    filter_doc = []
    for s,sent in enumerate(doc):
        filter_sent = sentences.filtersent(sent, remove_stopwords=True , stem=False, lemm=True, avoid_single_char=True)
        # Avoid null strings --> They cause problems
        if filter_sent == '':
            del doc[s]
        else:
            filter_doc.append(filter_sent)
        
    # feature summarise the passage
    select_indices = summarize.feature_summ(filter_doc, mode='sents', sent_lim=3, lam=0.7, mu=0.85)
    feature_base_sents.append([])
    for idx,selection in enumerate(select_indices):
        if selection==1:
            feature_base_sents[d].append(doc[idx])

    # Log progress
    if( d%10 == 0):
        print('#', end='')
    

###########################################################################

In [24]:
## Topic based summarisation

LSA_Gong_Liu_sents=[]
LSA_Stein_sents=[]
# LSA_Stein_MMR_Greed_sents=[]
LSA_Stein_MMR_ILP_sents=[]
LSA_Murray_sents=[]
LSA_Murray_CM_sents=[]
for d,doc in enumerate(sent_split_docs):
    
    filter_doc = []
    for s,sent in enumerate(doc):
        filter_sent = sentences.filtersent(sent, remove_stopwords=True , stem=True, lemm=True, avoid_single_char=True)
        # Avoid null strings --> They cause problems
        if filter_sent == '':
            del doc[s]
        else:
            filter_doc.append(filter_sent)
        
    # LSA Gong Liu topic summarise the passage
    select_indices = summarize.LSA_summ(filter_doc, selection='gong', sent_lim=3, mu=0.85, lam=0.7, cross_method=False)
    LSA_Gong_Liu_sents.append([])
    for idx,selection in enumerate(select_indices):
        if selection==1:
            LSA_Gong_Liu_sents[d].append(doc[idx])
            
    # LSA Stein topic summarise the passage
    select_indices = summarize.LSA_summ(filter_doc, selection='stein', sent_lim=3, mu=0.85, lam=0.7, cross_method=False)
    LSA_Stein_sents.append([])
    for idx,selection in enumerate(select_indices):
        if selection==1:
            LSA_Stein_sents[d].append(doc[idx])
            
    # LSA Stein MMR ILP topic summarise the passage
    select_indices = summarize.LSA_summ(filter_doc, selection='stein-ILP', sent_lim=3, mu=0.85, lam=0.7, cross_method=False)
    LSA_Stein_MMR_ILP_sents.append([])
    for idx,selection in enumerate(select_indices):
        if selection==1:
            LSA_Stein_MMR_ILP_sents[d].append(doc[idx])
            
    # LSA Murray topic summarise the passage
    select_indices = summarize.LSA_summ(filter_doc, selection='murray', sent_lim=3, mu=0.85, lam=0.7, cross_method=False)
    LSA_Murray_sents.append([])
    for idx,selection in enumerate(select_indices):
        if selection==1:
            LSA_Murray_sents[d].append(doc[idx])
            
    # LSA Murray topic summarise the passage
    select_indices = summarize.LSA_summ(filter_doc, selection='murray', sent_lim=3, mu=0.85, lam=0.7, cross_method=True)
    LSA_Murray_sents.append([])
    for idx,selection in enumerate(select_indices):
        if selection==1:
            LSA_Murray_sents[d].append(doc[idx])
            
    ### You can try other possibilities by enabling or disabling the cross method on different selection methods

    # Log progress
    if( d%10 == 0):
        print('#', end='')
    
    

###########################################################################

['Take, for e, the devices that make up SMB document management infrastructure inclusive of printers, scanners, copiers and that serve SMB document processing needs.',
 'But beyond just day-to-day device management, with managed print services, these devices communicate back to an operations center when critical supplies are needed to keep documents flowing within the organization.',
 'In addition to device management, ConnectKey opens document infrastructure to mobile users, allowing them to print and access data from their smartphones or tablets.']